## Theoretical Power

In [1]:

import itertools
import numpy as np


Define weights and quorum

In [2]:
## Number of votes necessary to approve initiative.
QUORUM = 10

## Weights of each player.
weights = [ 1, 5, 7, 3 ]
N = len(weights)

## Index of players (from 0 to N-1)
players = range(N)

#### Shapley Index

"The power of a coalition (or a player) is measured by the fraction of the possible voting sequences in which that coalition casts the deciding vote, that is, the vote that first guarantees passage or failure."

https://en.wikipedia.org/wiki/Shapley%E2%80%93Shubik_power_index

_Note:_ The key word is "sequences".

**Exercise: Compute the Shapley index for all players.**

_Note:_ `itertools` is your friend.

In [11]:
## Generate all possible permuations (i.e., coalitions).
perms = list(itertools.permutations(players))
perms

[(0, 1, 2, 3),
 (0, 1, 3, 2),
 (0, 2, 1, 3),
 (0, 2, 3, 1),
 (0, 3, 1, 2),
 (0, 3, 2, 1),
 (1, 0, 2, 3),
 (1, 0, 3, 2),
 (1, 2, 0, 3),
 (1, 2, 3, 0),
 (1, 3, 0, 2),
 (1, 3, 2, 0),
 (2, 0, 1, 3),
 (2, 0, 3, 1),
 (2, 1, 0, 3),
 (2, 1, 3, 0),
 (2, 3, 0, 1),
 (2, 3, 1, 0),
 (3, 0, 1, 2),
 (3, 0, 2, 1),
 (3, 1, 0, 2),
 (3, 1, 2, 0),
 (3, 2, 0, 1),
 (3, 2, 1, 0)]

In [ ]:
## Initialize array counting the number of times a player is pivotal.
pivotal = np.zeros(N)

for i in range(len(perms)):
    sum = 0
    
    for j in players:

        ## Sum all weights in sequence.
        player = perms[i][j] 
        sum += weights[player]

        ## As soon a player guarantees passing the quorum, 
        ## we mark it as pivotal and exit the loop.
        if sum >= QUORUM:
            pivotal[player] += 1
            break



In [4]:
## Compute Shapley power indexes per player.
pivotal / len(perms)

array([0.        , 0.16666667, 0.66666667, 0.16666667])

#### Banzhaf Index

"To calculate the power of a voter using the Banzhaf index, list all the winning coalitions, then count the critical voters. A critical voter is a voter who, if he changed his vote from yes to no, would cause the measure to fail."

https://en.wikipedia.org/wiki/Banzhaf_power_index

**Exercise: Compute the Banzhaf index for all players.**


In [5]:
## Generate all combinations of different length (>=2)
combs = []
for L in range(2, len(players) + 1):
    for subset in itertools.combinations(players, L):
        combs.append(subset)

In [6]:
combs

[(0, 1),
 (0, 2),
 (0, 3),
 (1, 2),
 (1, 3),
 (2, 3),
 (0, 1, 2),
 (0, 1, 3),
 (0, 2, 3),
 (1, 2, 3),
 (0, 1, 2, 3)]

In [7]:
## Initialize array counting the number of times a player is a swing voter.
swings = np.zeros(N)

## Counter for how many times a coalition is winning (> QUORUM)
winning_coalitions = 0

for i in range(len(combs)):
    
    ## First we need to find out if a coalition is winning.
    sum = 0
    comb = combs[i]
    for j in range(len(comb)):
        player = comb[j] 
        sum += weights[player]

    if sum >= QUORUM:
        winning_coalitions += 1

        ## For winning coalitions, we check all players
        ## wheather they are swing voters, i.e., if the
        ## coalition can reach the quorum without them.
        for j in range(len(comb)):
            player = comb[j]
            w = weights[player]
            if sum - w < QUORUM:
                swings[player] += 1



In [8]:
## The Banhzaf index for every player.
swings / winning_coalitions

array([0.        , 0.33333333, 1.        , 0.33333333])

## Empirical Power

Potential and exercised.
Based on "Voting Behavior and Power in Online Democracy: A Study of LiquidFeedback in Germany's Pirate Party" by Kling et al. 2015.

In [12]:
QUORUM = 10

## Here we have both weights and votes.
weights = [ 1, 5, 7, 3 ]
votes = [ 1, 1, 1, -1]

N = len(weights)
players = range(N)

n_infavor = votes.count(1)
n_against = votes.count(-1)
n_abstained = N - n_infavor - n_against

print(n_infavor, n_against, n_abstained)

## Summing total weights in favor (positive) 
## and against (negative).
Wp = 0
Wn = 0

for i in range(N):
    if votes[i] == 1: Wp += weights[i]
    elif votes[i] == -1: Wn += weights[i]

print(Wp, Wn)

3 1 0
13 3


#### Potential Power

Potential power tests if a weight _i_ is larger than the distance to quorum of all remaining votes:

It is equal to:

- 1, if test is positive;
- 0, if test is negative

**Exercise: Compute potential power for a given player.**

_Note_: in the paper they use quorum as a fraction, but you can use it as a count.

In [13]:
## Let's pick a player.
player = 0

vote_player = votes[player]
weight_player = weights[player]
vote_player_direction = (1 if vote_player > 0 else 0)

print(vote_player, weight_player, vote_player_direction)

1 1 1


In [14]:
distance_to_quorum_without_player = abs(QUORUM - (Wp - Wn) - (weight_player * vote_player_direction))
distance_to_quorum_without_player

1

In [15]:
potential_power = weight_player >= distance_to_quorum_without_player
potential_power = 1 if potential_power else 0
potential_power

1

#### Exercised Power

Exercised power tests if the outcome of a voting is different with and without a player _i_:

It is equal to:

- 1, if test is positive (outcomes differ);
- 0, if test is negative (same outcome)

**Exercise: Compute potential power for a given player.**

_Note_: in the paper they use quorum as a fraction, but you can use it as a count.

In [165]:
actual_voting_result = 1 if Wp - Wn >= QUORUM else 0
actual_voting_result

1

In [166]:
## Let's pick a player.
player = 0

vote_player = votes[player]
weight_player = weights[player]
vote_player_direction = (1 if vote_player > 0 else 0)


In [167]:
voting_without_player = Wp - (weight_player * vote_player_direction) - Wn > QUORUM
voting_without_player = 1 if voting_without_player else 0
voting_without_player

1

In [168]:
power_exercised = 1 if voting_without_player != actual_voting_result else 0
power_exercised

0